In [1]:
import pandas as pd
import numpy as np
import requests 
import pymysql
import toad
import joblib

In [31]:
def izi_list():
    conn = pymysql.connect(
        host="
    user='',
    password='',
        db="api_loansuper",
        charset="utf8")
    sql2 = f"""
SELECT
a.*,b.izidata_credit_feature_v2_json
from 
t_loan_order as a LEFT JOIN
t_app_full_link_info as b on a.loan_id=b.loan_id
WHERE
a.request_date>='2021-04-27'
and
a.request_date<='2021-05-15'
and
a.is_new=1
and
a.loan_order_status=5

    """
    df = pd.read_sql(sql2, conn)
    #df = df.drop_duplicates("app_user_id")
    conn.close()
    return df

In [32]:
data_izi=izi_list()

In [33]:
data_izi2=data_izi.drop_duplicates(subset='loan_id',keep='first')

In [34]:
import tqdm

In [35]:
ind=0
erro=[1]
for i in tqdm.tqdm(data_izi2.index):
    try:
        data1=pd.DataFrame(eval(data_izi2['izidata_credit_feature_v2_json'].loc[i]),index=[data_izi2['loan_id'].loc[i]])
        data1=data1[data1['loanId']==data_izi2['loan_id'].loc[i]]
        data1['repay_order_status']=data_izi2['repay_order_status'].loc[i]
        data1['request_date']=data_izi2['request_date'].loc[i]
        if ind==0:
            data_1=data1
            ind=1
        else:
            data_1=data_1.append(data1)
    except:
        erro.append(data_izi2['loan_id'].loc[i])
        continue

  0%|          | 0/14729 [00:00<?, ?it/s]/usr/local/lib64/python3.6/site-packages/pandas/core/frame.py:7138: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,
100%|██████████| 14729/14729 [12:59<00:00, 18.90it/s]


In [36]:
class cale_IV:
    def __init__(self, begin_date=None, end_date=None, data=None, col=None,cols=None):
#         self.begin_date = begin_date
#         self.end_date = end_date
        self.data = data
        self.col = col
        self.cols = cols
    def quantile_(self, data=None, col=None, bins_=10):
        quantile_list = [data[col].quantile(q * (1 / bins_)) for q in range(1, bins_)]
        bins = [-np.inf] + quantile_list + [np.inf]
        return sorted(list(set(bins)))

    def cut_bin(self, data=None, col=None, show_values=True):
        bins = self.quantile_(data=data, col=col, bins_=10)
        bins1 = bins[:-1]
        bins1[0] = "nagetive_inf"
        bins2 = bins[1:]
        bins2[-1] = "postive_inf"
        labels = [str(x) + "~" + str(y) for x, y in zip(bins1, bins2)]
        # print(labels)
        data[col + "_bin"] = pd.cut(data[col], bins=bins, labels=labels)
        if show_values:
            values_dict = {}
            #         values=data[col+"_bin"].value_counts(normalize=True).tolist()
            #         values_dict={"label_"+str(x):y for x,y in zip(labels,values)}
            values_dict["label"] = labels
            values=[str(i) for i in data[col + "_bin"].value_counts(normalize=True).tolist()]
            values_dict["values"] = values
            return values_dict
        else:
            return data[col + "_bin"]

    def cale_woe(self, df1, var, target):
        df = df1.copy()
        # df[var]=df[var].fillna("missing")

        data = df.groupby([var])[target].agg({
            "count": "count",
            "bad": "sum",
            "bad_rate": "mean"
        }).reset_index()
        bad_total = sum(data["bad"])
        data["bad_pcnt"] = data["bad"] / bad_total
        data["good"] = data["count"] - data["bad"]
        data["good_rate"] = 1 - data["bad_rate"]
        good_total = sum(data["good"])
        data["good_pcnt"] = data["good"] / good_total
        data["count_distr"] = data["count"] / df.shape[0]
        data["woe"] = np.log(data["good_pcnt"] / data["bad_pcnt"])
        data["iv"] = (data["good_pcnt"] - data["bad_pcnt"]) * np.log(
            data["good_pcnt"] / data["bad_pcnt"])
        data["iv_sum"] = sum(data["iv"])
        data["lift"] = data["bad_rate"] / df[target].mean()
        data["variable"] = var
        data = data[[
            "variable", var, 'count', 'bad', 'bad_rate', 'good', 'good_rate',
            'count_distr', 'woe', 'iv', 'iv_sum', 'lift'
        ]]
        return data
    def iv_dataframe(self, data=None,target="is_overdue", cols=[], show_dict=False):
        iv_data = pd.DataFrame()
        ivs = []

        for col in cols:
            bins = (self.quantile_(data=data, col=col, bins_=6))
            data[col + "_range"] = pd.cut(data[col],
                                          bins=bins)
            iv_ = self.cale_woe(data, col + "_range", target).iv_sum[0]
            ivs.append(iv_)
        iv_data["columns"] = cols
        iv_data["iv"] = ivs
        iv_data = iv_data.sort_values(by="iv", ascending=False)

        if show_dict:
            iv_data={x:str(y) for x ,y in zip(iv_data["columns"].tolist(),iv_data["iv"].tolist())}
   
        return iv_data

In [40]:
data_1['repay_order_status']=1-data_1['repay_order_status']

In [42]:
cale_IV().iv_dataframe(data_1,target='repay_order_status',cols=gg[1:])[0:50]

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:40: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version. Use                 named aggregation instead.

    >>> grouper.agg(name_1=func_1, name_2=func_2)

/usr/local/lib64/python3.6/site-packages/pandas/core/series.py:856: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


,columns,iv
28,bIi7d,0.124807
26,bIi3d,0.119279
40,bPi7d,0.110687
21,bIi14d,0.107850
23,bIi21d,0.105578
33,bPi14d,0.101299
38,bPi3d,0.098740
35,bPi21d,0.097938
24,bIi30d,0.095233
76,idinquiries7d,0.091539
